In [ ]:
import sys 
sys.path.append("../")

import nerf_model
import dataloader
import nerf_helpers

import cv2
from PIL import Image

import gc
import torch
import itertools
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Visualizing Datasets

In [ ]:
def visualize(coords, rgb): 
    a, b = coords.shape
    if b == 3:
        coords = coords.T
    if type(rgb) != str:
        a, b = rgb.shape
        if a == 3:
            rgb = rgb.T
    plot_fig = go.Scatter3d(x=coords[0], y=coords[1], z=coords[2], 
    mode='markers', marker=dict(
       size=2,
       color=rgb
    ),)
    return plot_fig

def line_visualize(coords, rgb): 
    a, b = coords.shape
    if b == 3:
        coords = coords.T
    if type(rgb) != str:
        rgb = rgb.T
    plot_fig = go.Scatter3d(x=coords[0], y=coords[1], z=coords[2], 
    mode='lines', line=dict(
       width=1,
       color=rgb
    ),)
    return plot_fig

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for i, batch in enumerate(iter(sdl)):
    if i < 5:
        continue
    gc.collect()
    nerf_helpers.fix_batchify(batch)
    origins = batch['all_origin'].reshape((-1,3))[::10,:]
    direcs = batch['all_direc'].reshape((-1, 3))[::10,:]
    images = batch['image'].reshape((-1, 3))[::10,:]
    im_coords = origins + 6 * direcs
    pic = visualize(im_coords, images)
    o = visualize(origins[0,None,:], 'purple')
    fig.add_trace(pic)
    fig.add_trace(o)
fig.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for i, batch in enumerate(iter(sdl)):
    if i == 2:
        break
    gc.collect()
    nerf_helpers.fix_batchify(batch)
    origins = batch['all_origin'][200:600,200:600,:].reshape((-1,3))[::300,:]
    direcs = batch['all_direc'][200:600,200:600,:].reshape((-1, 3))[::300,:]
    images = batch['image'][200:600,200:600,:].reshape((-1, 3))[::300,:]
    
    samples, ts = nerf_helpers.generate_coarse_samples(origins, direcs, 16)
    rgb = torch.broadcast_to(images[:, None, :], samples.shape)
    rgb = rgb.reshape((-1,3)).T
    samples = samples.view((-1,3))
    pic = line_visualize(samples, rgb)
    o = visualize(origins[0,None,:], 'purple')
    fig.add_trace(pic)
    fig.add_trace(o)
fig.show()

In [ ]:
def inverse_transform_sampling(o_rays: torch.Tensor, d_rays: torch.Tensor, weights, ts, num_samples):
    """Performs inverse transform sampling according to the weights.

    Samples from ts according to the weights (i.e. ts with higher weights are 
    more likely to be sampled).
    
    Probably not the best implementation, since the official NeRF implementation 
    does something different. This is probably good enough though? Good thing
    I don't have to be rigorous. 

    Args:
        o_rays: [N x 3] coordinates of the ray origin.
        d_rays: [N x 3] directions of the ray.
        weights: [N x C x 1] tensor of weights calculated as 
                 w = T(1 - exp(- density * delta)). N is the batch size, and C 
                 is the number of coarse samples.
        ts: [N x C x 1] is the increment between each sample. N is the batch 
            size, and C is the number of coarse samples. 
        num_samples: number of samples to return per ray.
    Returns:
        fine_samples: [N x num_samples x 3] tensor sampled according to weights.
                      Instead of using the same values as in ts, we pertube it by 
                      adding random noise (sampled from U(0, 1/num_samples)).
        fine_ts: [N x num_samples x 1] tensor of the time increment for each sample. 
    """
    N, C, _ = ts.shape
    o_rays = o_rays.unsqueeze(1)
    d_rays = d_rays.unsqueeze(1)

    cdf = torch.cumsum(weights, axis=1)  # [N x C x 1]
    cdf = cdf / cdf[:, -1, None]
    eps = torch.rand((N, 1), device=device) / num_samples  # low variance sampling
    samples = torch.arange(0, 1, 1 / num_samples, device = device)
    samples = torch.broadcast_to(samples, (N, num_samples))
    samples = samples + eps

    cdf = torch.squeeze(cdf, -1)  # make dimensions match, [N x C]
    idxs = torch.searchsorted(cdf, samples).unsqueeze(-1)  # [N x C x 1]
    idxs[idxs >= C] = C - 1
    bins = torch.gather(ts, 1, idxs)
    
    fine_ts = bins + 2 * torch.rand((N, num_samples, 1), device=device) / C # num_samples
    fine_samples = o_rays + fine_ts * d_rays
    return fine_samples, fine_ts

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

coarse_samples, coarse_ts = nerf_helpers.generate_coarse_samples(o_rays, d_rays, 64, 2., 6.)
weights = torch.arange(1, 65, 1).view(coarse_ts.shape)

num_samples = 128
fine_samples, fine_ts = sample_pdf(o_rays, d_rays, weights, coarse_ts, num_samples)

print(fine_samples.shape)
# print(fine_samples)
pic = visualize(fine_samples.view((-1,3)), 'black')
o = visualize(o_rays[0,None,:], 'purple')

fig.add_trace(pic)
fig.add_trace(o)
fig.show()
# print(fine_ts)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

coarse_samples, coarse_ts = nerf_helpers.generate_coarse_samples(o_rays, d_rays, 64, 2., 6.)
weights = torch.arange(1, 65, 1).view(coarse_ts.shape)

num_samples = 128
fine_samples, fine_ts = nerf_helpers.inverse_transform_sampling(o_rays, d_rays, weights, coarse_ts, num_samples)

print(fine_samples.shape)
# print(fine_samples)
pic = visualize(fine_samples.view((-1,3)), 'black')
cpic = visualize(coarse_samples.view((-1,3)), 'red')
o = visualize(o_rays[0,None,:], 'purple')

fig.add_trace(pic)
fig.add_trace(cpic)
fig.add_trace(o)
fig.show()
# print(fine_ts)

In [ ]:
fig = go.Figure(go.Histogram(x=fine_ts.view((-1,))))
fig.show()

### Model Debugging

In [ ]:
from importlib import reload
dataloader = reload(dataloader)
nerf_model = reload(nerf_model)
nerf_helpers = reload(nerf_helpers)
gc.collect()

In [ ]:
base_dir = '../data/lego/'
sdl = dataloader.getSyntheticDataloader(base_dir, 'val', 1, num_workers=1, shuffle=True)

batch = next(iter(sdl))
nerf_helpers.fix_batchify(batch)
print(batch.keys())
print(batch['rgb'].shape)
print(batch['origin'].shape)
print(batch['direc'].shape)
print(batch['xs'].shape)

o_rays = batch['origin']
d_rays = batch['direc']
rgb = batch['rgb']
xs = batch['xs']
ys = batch['ys']

In [ ]:
network = nerf_model.NeRFNetwork(position_dim=10, direction_dim=4, coarse_samples=64,
                 fine_samples=128)

In [ ]:
network.train()
pred_dict = network.forward(o_rays, d_rays)
fine_rgbs = pred_dict['fine_rgb_rays']
coarse_rgbs = pred_dict['coarse_rgb_rays']

fine_loss = F.mse_loss(fine_rgbs, rgb)
coarse_loss = F.mse_loss(coarse_rgbs, rgb)
loss = coarse_loss + fine_loss
loss.backward()
print(loss)

In [ ]:
pred_dict.keys()

In [ ]:
pred_dict['coarse_ts']

In [ ]:
pred_dict['coarse_deltas']

In [ ]:
fc = torch.nn.Linear(2,2)

In [ ]:
multi = torch.Tensor([[[1.,2.],[3.,4.]]])
single = torch.Tensor([[1.,2.],[3.,4.]])

In [ ]:
fc(multi)

In [ ]:
fc(single)